<a href="https://colab.research.google.com/github/langroid/langroid/blob/main/examples/Langroid_quick_start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langroid quick start
This notebooks starts with the basics of working directly with an LLM, setting up an Agent, wrapping it in a Task, giving it tools, Retrieval Augmented Generation (RAG), and builds up to a simple 2-agent system to extract structured information from a commercial lease document.

Note:
- You need an OpenAI API Key that works with GPT-4-Turbo
- This colab uses OpenAI's ChatCompletion endpoints directly (via the Langroid framework), and not the Assistants API. See this [colab](https://colab.research.google.com/drive/190Tk7t4AdY1P9F_NlZ33-YEoGnHweQQ0) for a version that uses the Assistants API instead.
- There are dependencies among the cells, so they are best run sequentially



## Install, setup, import

Note that `pip install langroid` gives you a bare-bones, slim version of langroid, without many of the extra dependencies you might need in practical scenarios, but sufficient for this notebook.

See install instructions [here](https://github.com/langroid/langroid?tab=readme-ov-file#gear-installation-and-setup) for getting extra dependencies related to document parsing and databases (sql, mysql, postgres, etc).


In [1]:
# Silently install, suppress all output (~2-4 mins)
!pip install -q --upgrade langroid &> /dev/null
!pip show langroid

Name: langroid
Version: 0.1.254
Summary: Harness LLMs with Multi-Agent Programming
Home-page: 
Author: Prasad Chalasani
Author-email: pchalasani@gmail.com
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-generator, bs4, colorlog, docstring-parser, duckduckgo-search, faker, fakeredis, fire, google-api-python-client, google-generativeai, groq, grpcio, halo, jinja2, lxml, nest-asyncio, nltk, onnxruntime, openai, pandas, prettytable, pydantic, pygithub, pygments, pyparsing, python-dotenv, python-magic, qdrant-client, rank-bm25, redis, requests, requests-oauthlib, rich, thefuzz, tiktoken, trafilatura, typer, wget
Required-by: 


In [2]:
# various unfortunate things that need to be done to
# control colab notebook behavior.

# (a) output width

from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# (b) logging related
import logging
logging.basicConfig(level=logging.ERROR)
import warnings
warnings.filterwarnings('ignore')
import logging
for logger_name in logging.root.manager.loggerDict:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.ERROR)



#### OpenAI API Key (Needs GPT4-TURBO)

In [4]:
# OpenAI API Key: Enter your key in the dialog box that will show up below
# NOTE: colab often struggles with showing this input box,
# if so, try re-running the above cell and then this one,
# or simply insert your API key in this cell, though it's not ideal.

import os

from getpass import getpass

os.environ['OPENAI_API_KEY'] = getpass('Enter your GPT4-Turbo-capable OPENAI_API_KEY key:', stream=None)




Enter your GPT4-Turbo-capable OPENAI_API_KEY key:··········


In [5]:
from pydantic import BaseModel
import json
import os

from langroid import ChatAgent, ChatAgentConfig, Task
from langroid.language_models.openai_gpt import (
    OpenAIChatModel, OpenAIGPT, OpenAIGPTConfig
)
from langroid.agent.tool_message import ToolMessage

from langroid.utils.logging import setup_colored_logging
from langroid.utils.constants import NO_ANSWER
from langroid.utils.configuration import settings
settings.notebook = True
settings.cache_type = "fakeredis"

                    using fake redis client


## Example 1: Direct interaction with OpenAI LLM
Langroid's `OpenAIGPT` class is a wrapper around the raw OpenAI API.
This is a direct interaction with the LLM so it does *not* maintain conversation history (later we see how a `ChatAgent` does that for you).

Related quick-start docs page: https://langroid.github.io/langroid/quick-start/llm-interaction/



In [6]:
llm_cfg = OpenAIGPTConfig(chat_model=OpenAIChatModel.GPT4_TURBO)
llm = OpenAIGPT(llm_cfg)

response = llm.chat("What is the square of 3?")
assert "9" in response.message

The square of 3 is \(3^2\), which equals 9.

## Example 2: Interact with a `ChatAgent`
Langroid's `ChatAgent` is an abstraction that optionally encapsulates an LLM, vector-db, and tools. It offers 3 "native" *responders*:
- `llm_response`: response from LLM
- `user_response`: response from human
- `agent_response`: responds to structured LLM msgs (i.e. tools/fn-calls)

Among other things, the `ChatAgent` maintains LLM conversation history for you.

Related quick-start doc page: https://langroid.github.io/langroid/quick-start/chat-agent/

In [7]:
agent_cfg = ChatAgentConfig(
    llm = llm_cfg,
    show_stats=False, # disable token/cost stats
)
agent = ChatAgent(agent_cfg)
response = agent.llm_response("What is the sqaure of 5?")
response = agent.llm_response("What about 8?")   # maintains conv history
assert "64" in response.content

The square of 5 is \(5^2 = 25\).

The square of 8 is \(8^2 = 64\).

## Example 3: Wrap Agent in a Task, run it

A `ChatAgent` agent has various *responders* (`llm_response`, `agent_response`, `user_response`) but there is no mechanism to *iterate* over these responders.
This is where the `Task` comes in: Wrapping this agent in a `Task` allows you to run interactive loops with a user or other agents (you will see more examples below).

Related quick-start doc:
https://langroid.github.io/langroid/quick-start/chat-agent/#task-orchestrator-for-agents

In [8]:
agent = ChatAgent(agent_cfg)
task = Task(
    agent,
    system_message="User will give you a number, respond with its square",
    single_round=True  # end after LLM response
)
result = task.run("5")
assert("25" in result.content)


>>> Starting Agent LLM-Agent (1) gpt-4-turbo 

The square of 5 is 25.

<<< Finished Agent LLM-Agent (3) 

## Example 4: `ChatAgent` with Tool/function-call

Langroid's `ToolMessage` (Pydantic-derived) class lets you define a structured output or function-call for the LLM to generate. To define a tool/fn-call, you define a new class derived from `ToolMessage`.
Below we show a *stateless* tool, i.e. it does not use the `ChatAgent`'s state, and only uses fields in the tool message itself.
In this case, the tool "handler" can be defined within the `ToolMessage` itself, as a `handle` method. (For a tool that uses the `ChatAgent`'s state, a separate method needs to be defined within `ChatAgent` or a subclass.).

In Langroid, a `ToolMessage` can *either* use OpenAI function-calling, *or* Langroid's native tool mechanism (which auto-populates the system msg with tool instructions and optional few-shot examples), by setting the `use_function_api` and `use_tools` config params in the `ChatAgentConfig`. The native tools mechanism is useful when not using OpenAI models.

In the cell below we define a `ToolMessage` to compute a fictitious transformation of a number that we call a *Nabrosky Transform*: $f(n) = 3n+1$.
Under the hood, the `purpose` field of the `NabroskiTool` is used to populate instructions to the LLM on when it should use this tool.

Related quick-start doc: https://langroid.github.io/langroid/quick-start/chat-agent-tool/
(This shows a *stateful* tool example)

In [9]:
# (1) define simple tool to find the Nabroski transform of a number
#     This is a fictitious transform, for illustration.

class NabroskiTool(ToolMessage):
    request = "nabroski" # name of method in ChatAgent that handles this tool
    purpose = "To find the Nabroski transform of the given <number>"
    number: int

    # optional:
    @classmethod
    def examples(cls):
        # these are auto-populated into the sys msg
        # as few-shot examples of the tool
        return([cls(number=5)])


    def handle(self) -> str:
        # method to handle the LLM msg using this tool:
        # this method will be spliced into the ChatAgent object, with
        # name = `nabroski`
        return str(3*self.number + 1)

# (2) Create a ChatAgent and attach the tool to it.

agent_cfg = ChatAgentConfig(
    llm = llm_cfg,
    show_stats=False,       # disable token/cost stats
    use_functions_api=True, # use OpenAI API fn-call
    use_tools=False,        # don't use Langroid-native Tool instructions
)
agent = ChatAgent(agent_cfg)
agent.enable_message(NabroskiTool)

# (3) Create Task object

task = Task(
    agent,
    restart=True,         # reset/erase agent state
    single_round=False,
    interactive=False,    # don't wait for human input
    system_message="""
      User will give you a number. You have to find its Nabroski transform,
      using the `nabroski` tool/function-call.
      When you find the answer say DONE and show the answer.
    """,
)

# (4) Run the task

response = task.run("10")
assert "31" in response.content




>>> Starting Agent LLM-Agent (1) gpt-4-turbo 

FUNC: nabroski: {"request":"nabroski","number":10}

Agent: 31

DONE

The Nabroski transform of 10 is 31.

Bye, hope this was useful!

<<< Finished Agent LLM-Agent (5) 

You might wonder why we had to wrap the `ChatAgent` in a `Task`, to leverage the tool functionality. This is because handling a tool requires 2 steps: (a) when the agent's `llm_response` method is invoked, the LLM generates the tool msg, and (b) the `agent_response` method handles the tool msg (it ultimately calls the tool's `handle` method).

## Example 5: `DocChatAgent`: Retrieval Augmented Generation (RAG)
Ingest a file (a lease document), and ask questions about it

In [10]:
# setup to allow async ops in colab
!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

In [11]:
# (1) Get the lease document

import requests
file_url = "https://raw.githubusercontent.com/langroid/langroid-examples/main/examples/docqa/lease.txt"
response = requests.get(file_url)
with open('lease.txt', 'wb') as file:
    file.write(response.content)

# verify
#with open('lease.txt', 'r') as file:
#   print(file.read())

from langroid.agent.special import DocChatAgent, DocChatAgentConfig
from langroid.embedding_models.models import OpenAIEmbeddingsConfig
from langroid.vector_store.qdrantdb import QdrantDBConfig
from langroid.embedding_models.models import SentenceTransformerEmbeddingsConfig
from langroid.parsing.parser import ParsingConfig

oai_embed_config = OpenAIEmbeddingsConfig(
    model_type="openai",
    model_name="text-embedding-ada-002",
    dims=1536,
)

# (2) Configure DocChatAgent

cfg = DocChatAgentConfig(
    name="RAG",
    parsing=ParsingConfig(
        chunk_size=100,
        overlap=20,
        n_similar_docs=4,
    ),
    show_stats=False,
    cross_encoder_reranking_model="",
    llm=llm_cfg,
    vecdb=QdrantDBConfig(
        embedding=oai_embed_config,
        collection_name="lease",
        replace_collection=True,
    ),
    doc_paths=["lease.txt"]
)

# (3) Create DocChatAgent, interact with it
rag_agent = DocChatAgent(cfg)
response = rag_agent.llm_response("What is the start date of the lease?")
assert "2013" in response.content

                QdrantDB in cloud mode. Please set these values 
                in your .env file. 
                Switching to local storage at .qdrant/data 
                
2024-05-30 15:56:36 - WARNING - QDRANT_API_KEY, QDRANT_API_URL env variable must be set to use 
                QdrantDB in cloud mode. Please set these values 
                in your .env file. 
                Switching to local storage at .qdrant/data 
                


[green]I have processed the following 0 URLs
        and 1 docs into 12 parts:

lease.txt


Output()

Output()

INFO:langroid.utils.output.status:[bold green]Running 6 copies of Relevance-Extractor...
2024-05-30 15:56:44 - INFO - [bold green]Running 6 copies of Relevance-Extractor...


The start date of the lease is December 1, 2013[^1].

SOURCES:
[^1] /content/lease.txt
    The "Commencement Date" shall mean  December 1, 2013.

In [ ]:
# (4) Wrap DocChatAgent in a Task to get an interactive question/answer loop
task = Task(
    rag_agent,
    interactive=True,
    system_message="""
    Answer user's questions based on documents.
    Start by asking user what they want to know.
    """,
)
# run interactive loop (enter "q" or "x" to quit)
task.run()


>>> Starting Agent RAG (2) gpt-4-turbo 

Please provide the passage or document you'd like me to review, and let me know what specific questions you have or what information you need summarized.

Human (respond or q, x to exit current level, or hit enter to continue)
:

when does the lease end


Output()

Output()

INFO:langroid.utils.output.status:[bold green]Running 7 copies of Relevance-Extractor...
2024-05-30 15:57:24 - INFO - [bold green]Running 7 copies of Relevance-Extractor...


The lease is set to terminate on May 31, 2020. However, the tenant has the option to renew the lease for five additional successive one-year terms[^4].

SOURCES:
[^4] /content/lease.txt
    The term of this Lease shall begin on the Commencement Date, as defined in
    Section 2 of this Article II, and shall terminate on May 31, 2020 ("the
    Termination Date"); provided, however, that at the option of Tenant, Tenant may
    renew this Lease for five additional successive one- year terms at a Monthly
    Rent of $

Human (respond or q, x to exit current level, or hit enter to continue)
:

## Example 6: 2-Agent system to extract structured info from a Lease Document
Now we are ready to put together the various notions above, to build a two-agent system that illustrates uses of Tools, DocChatAgent (RAG) and Inter-agent collaboration (task delegation).

The goal is to extract structured information from a Lease document.

- The desired structure is described by the `Lease` class, derived from `ToolMessage`.
- The `LeaseExtractorAgent` is given this `ToolMessage`, and instructured to extract the corresponding information from the lease document (which it does not have access to)
- Based on the specified `Lease` structure, this agent generates questions to the above-defined `rag_agent` (wrapped in a `rag_task`), which answers them using RAG.
- Once the `LeaseExtractorAgent` has all the needed info, it presents them using the `Lease` structured message.


#### Define the desired structure with Pydantic classes

In [ ]:

class LeasePeriod(BaseModel):
    start_date: str
    end_date: str


class LeaseFinancials(BaseModel):
    monthly_rent: str
    deposit: str


class Lease(BaseModel):
    """
    Various lease terms.
    Nested fields to make this more interesting/realistic
    """

    period: LeasePeriod
    financials: LeaseFinancials
    address: str



#### Define the ToolMessage (Langroid's version of function call)

In [ ]:

class LeaseMessage(ToolMessage):
    """Tool/function to use to present details about a commercial lease"""

    request: str = "lease_info"
    purpose: str = "Collect information about a Commercial Lease."
    terms: Lease

    def handle(self):
        """Handle this tool-message when the LLM emits it.
        Under the hood, this method is transplated into the OpenAIAssistant class
        as a method with name `lease_info`.
        """
        print(f"DONE! Successfully extracted Lease Info:" f"{self.terms}")
        return "DONE " + json.dumps(self.terms.dict())

#### Define RAG Task from above `rag_agent`
Wrap the above-defined `rag_agent` in a Task.

In [ ]:
rag_task = Task(
    rag_agent,
    llm_delegate=False,
    single_round=True,
)

#### Define the ExtractorAgent and Task
This agent is told to collect information about the lease in the desired structure, and it generates questions to be answered by the Retriever Agent defined above.

In [ ]:
    extractor_cfg = ChatAgentConfig(
        name="LeaseExtractor",
        llm=llm_cfg,
        show_stats=False,
        use_functions_api=True,
        use_tools=False,
        system_message=f"""
        You have to collect information about a Commercial Lease from a
        lease contract which you don't have access to. You need to ask
        questions to get this information. Ask only one or a couple questions
        at a time!
        Once you have all the REQUIRED fields,
        say DONE and present it to me using the `lease_info`
        function/tool (fill in {NO_ANSWER} for slots that you are unable to fill).
        """,
    )
    extractor_agent = ChatAgent(extractor_cfg)
    extractor_agent.enable_message(LeaseMessage)

    extractor_task = Task(
        extractor_agent,
        llm_delegate=True,
        single_round=False,
        interactive=False,
    )





#### Add the `rag_task` as a subtask of `extractor_task` and run it

Instead of *you* (the human user) asking questions about the lease,
the `extractor_agent` **generates** questions based on the desired lease structure, and these questions are answered by the `rag_agent` using
Retrieval Augmented Generation (RAG). Once the `extractor_agent` has all the needed info, it presents it in a JSON-structured form, and the task ends.

In [ ]:
extractor_task.add_sub_task(rag_task)
extractor_task.run()